# Py05 - Graph Traversals

## Overview

A graph consists of vertices and edges. Vertices are stored as documents in vertex collections, and edges are stored as documents in edge collections. The collections used in a graph and their relations are specified with edge definitions.

In this tutorial we will build a simple application for people to find new friends called MacroFriends, and then use graph traversals to see who would like to meet whom.

## Pre-requisite

Let's assume your 

- Tenant name is an email address
- User password is xxxxx

If you need to install pyc8, you can run the cell below; otherwise, you may skip it.

In [ ]:
!pip install pyC8

## 1. Importing Libraries & Define Variables

In [ ]:
from c8 import C8Client
import pprint
import json

# Variables - Queries
global_url = "gdn.paas.macrometa.io"
email = "email" # <-- Email goes here
password = "password" # <-- password goes here
geo_fabric = "_system"
collection_friends = "Friends"
collection_likes = "Likes"

collection_graph = "Friends_graph"
read_friends = "FOR friends IN Friends RETURN friends"
read_likes = "FOR likes IN Likes RETURN like"

MacroFriends_Client = "SanJoseLou"
# MacroFriends_Client = "StocktonPeria"


graph_traversal1 = "FOR c IN Friends FILTER c.macrofriend_user_name == \"{}\" FOR v IN 1..1 INBOUND c Likes RETURN v".format(MacroFriends_Client)

graph_traversal2 = "FOR c IN Friends FILTER c.macrofriend_user_name == \"{}\" FOR v IN 1..1 OUTBOUND c Likes RETURN v".format(MacroFriends_Client)

pp = pprint.PrettyPrinter(indent=4)

## 2. Connecting to GDN

In [ ]:
print("\n ------- CONNECTION SETUP  ------")
print("tenant: {}, geofabric:{}".format(email, geo_fabric))
client = C8Client(protocol='https', host=global_url, port=443,
                email=email, password=password,
                geofabric=geo_fabric)    

## 3. Creating Collections

In [ ]:
# Create collections if not exists
print("2a. CREATE_FRIENDS_VERTEX_COLLECTION: region: {},  collection: {}".format(global_url, collection_friends))
if client.has_collection(collection_friends):
    friendsCol = client.collection(collection_friends)
else:
    friendsCol = client.create_collection(collection_friends)

print("2b. CREATE_LIKES_EDGE_COLLECTION: region: {},  collection: {}".format(global_url, collection_likes))
if client.has_collection(collection_likes):
    likesCol = client.collection(collection_likes)
else:
    likesCol = client.create_collection(collection_likes, edge=True)

## 4. Inserting Data - "Friends"

In [ ]:
# Insert data into collections.
print("3a. INSERT_FRIENDS_DATA: region: {}, collection: {}".format(global_url, collection_friends))

docs = [
  {
    "full_name": "Kiel Dummer",
    "first_name": "Kiel",
    "last_name": "Dummer",
    "city": "Burbank",
    "state": "CA",
    "address": "40317 5th Crossing",
    "macrofriend_user_name": "BurbankKeil",
    "_key": "BurbankKeil"
  },
  {
    "full_name": "Pernell Winspare",
    "first_name": "Pernell",
    "last_name": "Winspare",
    "city": "San Diego",
    "state": "CA",
    "address": "596 Packers Pass",
    "macrofriend_user_name": "SanDiegoPernell",
    "_key": "SanDiegoPernell"
  },
  {
    "full_name": "Ava Kermath",
    "first_name": "Ava",
    "last_name": "Kermath",
    "city": "Berkeley",
    "state": "CA",
    "address": "2 Doe Crossing Junction",
    "macrofriend_user_name": "BerkeleyAva",
    "_key": "BerkeleyAva"
  },
  {
    "full_name": "Tremain McGrah",
    "first_name": "Tremain",
    "last_name": "McGrah",
    "city": "Torrance",
    "state": "CA",
    "address": "079 Russell Street",
    "macrofriend_user_name": "TorranceTremain",
    "_key": "TorranceTremain"
  },
  {
    "full_name": "Vidovik Boddam",
    "first_name": "Vidovik",
    "last_name": "Boddam",
    "city": "Los Angeles",
    "state": "CA",
    "address": "3 Brentwood Crossing",
    "macrofriend_user_name": "LosAngelesVidovik",
    "_key": "LosAngelesVidovik"
  },
  {
    "full_name": "Oralie Goward",
    "first_name": "Oralie",
    "last_name": "Goward",
    "city": "Los Angeles",
    "state": "CA",
    "address": "922 Columbus Park",
    "macrofriend_user_name": "LosAngelesOralie",
    "_key": "LosAngelesOralie"
  },
  {
    "full_name": "Lou Feaveer",
    "first_name": "Lou",
    "last_name": "Feaveer",
    "city": "San Jose",
    "state": "CA",
    "address": "55223 Hooker Crossing",
    "macrofriend_user_name": "SanJoseLou",
    "_key": "SanJoseLou"
  },
  {
    "full_name": "Peria King",
    "first_name": "Peria",
    "last_name": "King",
    "city": "Stockton",
    "state": "CA",
    "address": "8 Troy Plaza",
    "macrofriend_user_name": "StocktonPeria",
    "_key": "StocktonPeria"
  }
]
friendsCol.insert_many(docs)

## 5. Inserting Data - "Who Likes Whom"

In [ ]:
print("3a. INSERT_LIKES_DATA: region: {}, collection: {}".format(global_url, collection_likes))
docs = [
        {
    "macrofriend_user_name": "BurbankKeil",
    "friends_liked": "SanJoseLou",
    "_from": "Friends/BurbankKeil",
    "_to": "Friends/SanJoseLou",
    "liked_date": "1/4/2020",
    "liked_time": "13:33",
    "liked_amount": 30,
  },
  {
    "macrofriend_user_name": "SanJoseLou",
    "friends_liked": "StocktonPeria",
    "_from": "Friends/SanJoseLou",
    "_to": "Friends/StocktonPeria",
    "liked_date": "1/28/2020",
    "liked_time": "3:02",
    "liked_amount": 18,
  },
  {
    "macrofriend_user_name": "BerkeleyAva",
    "friends_liked": "StocktonPeria",
    "_from": "Friends/BerkeleyAva",
    "_to": "Friends/StocktonPeria",
    "liked_date": "1/28/2020",
    "liked_time": "3:02",
    "liked_amount": 18,
  }
    ]
likesCol.insert_many(docs)

## 6. Creating a Graph

In [ ]:
#Create a graph
print("4. CREATE_GRAPH...Friends_graph")
graph = client.create_graph(collection_graph)
register = graph.create_edge_definition(
        edge_collection=collection_likes,
        from_vertex_collections=[collection_friends],
        to_vertex_collections=[collection_friends]
    )

## 7. Execute a Traversal - "Find Friends who who would like to meet Lou?"

In [ ]:
#Read Data
print("5a. GRAPH_TRAVERSAL: Find Friends who would like to meet: {}".format(MacroFriends_Client))
cursor = client.execute_query(graph_traversal1)
docs = [document for document in cursor]
pp.pprint(docs)

## 8. Execute a Traversal - "Find Friends who Lou would like to meet?"

In [ ]:
print("5b. GRAPH_TRAVERSAL: Find Friends who were liked by Lou: {}".format(MacroFriends_Client))
cursor = client.execute_query(graph_traversal2)
docs = [document for document in cursor]
pp.pprint(docs)

## 9. Delete Data and Collections

In [ ]:
#Delete Data
print("6. DELETE_DATA...")
likesCol.truncate()
friendsCol.truncate()
client.delete_graph(collection_graph, drop_collections=True)

## Section Completed!

Congratulations! You have completed this tutorial.